In [1]:
from __future__ import division, print_function
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
plt.rcParams['image.cmap'] = 'gist_earth'
np.random.seed(98765)

In [2]:
import tensorflow as tf
tf.test.is_gpu_available()
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
tf.test.gpu_device_name()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12926808748623440780
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15862523495
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15613354397999890952
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:0d:00.0, compute capability: 6.0"
]


'/device:GPU:0'

In [13]:
import sys
# Add the tf_unet folder path to the sys.path list
sys.path.append('/data3/floraxue/cs282a/SegmentNet/')


In [14]:
from tf_unet import image_gen
from tf_unet import unet
from tf_unet import util, image_util

In [15]:
#Data input
#put your data here
path_images = "/data3/floraxue/cs282a/data/dataset_1/trainval/C_phase_full/"
generator = image_util.ImageDataProvider(path_images+"*.tif")
#TODO: need to enhance the borders and rethink the matrix vs bkg 

Number of files used: 100
Number of channels: 1
Number of classes: 2


In [18]:
net = unet.Unet(channels=1, n_class=2, layers=3, features_root=16)

2019-05-04 19:48:59,511 Layers 3, features 16, filter size 3x3, pool size: 2x2


In [19]:
trainer = unet.Trainer(net, batch_size=1, verification_batch_size = 1, 
                       optimizer="adam", opt_kwargs=dict(learning_rate = 0.0001))

In [20]:
path = trainer.train(generator, "./unet_C_adam_full", training_iters=100, epochs=3, display_step=40)

2019-05-04 19:49:01,316 Removing '/data3/floraxue/cs282a/SegmentNet/C_net/prediction'
2019-05-04 19:49:01,410 Removing '/data3/floraxue/cs282a/SegmentNet/C_net/unet_C_adam_full'
2019-05-04 19:49:01,485 Allocating '/data3/floraxue/cs282a/SegmentNet/C_net/prediction'
2019-05-04 19:49:02,993 Verification error= 82.3%, loss= 0.7124
2019-05-04 19:49:06,102 Start optimization
2019-05-04 19:49:11,769 Iter 0, Minibatch Loss= 0.6963, Training Accuracy= 0.4316, Minibatch error= 56.8%
2019-05-04 19:50:01,923 Iter 40, Minibatch Loss= 0.6775, Training Accuracy= 0.7574, Minibatch error= 24.3%
2019-05-04 19:50:52,360 Iter 80, Minibatch Loss= 0.6905, Training Accuracy= 0.4801, Minibatch error= 52.0%
2019-05-04 19:51:14,459 Epoch 0, Average loss: 0.6720, learning rate: 0.0001
2019-05-04 19:51:15,331 Verification error= 17.7%, loss= 0.5176
2019-05-04 19:51:46,571 Iter 120, Minibatch Loss= 0.5781, Training Accuracy= 0.3540, Minibatch error= 64.6%
2019-05-04 19:52:37,147 Iter 160, Minibatch Loss= 0.2719, 

In [ ]:
from skimage import io
x_test = io.imread(path_images+'0000.tif')
y_test = io.imread(path_images+'0000_mask.tif')

In [ ]:
fig, ax = plt.subplots(1,2, sharey=True, figsize=(10,5))
ax[0].imshow(x_test, aspect="auto")
ax[1].imshow(y_test, aspect="auto")

In [ ]:
x_test.shape

In [ ]:
#x_test, y_test = generator(1)
h,w = x_test.shape
x = x_test.reshape(1,h,w,1)
prediction = net.predict("./unet_trained/model.ckpt", x)

In [ ]:
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(12,5))
ax[0].imshow(x_test, aspect="auto")
ax[1].imshow(y_test, aspect="auto")
mask = prediction[0,...,1] > 0.1
ax[2].imshow(mask, aspect="auto")
ax[0].set_title("Input")
ax[1].set_title("Ground truth")
ax[2].set_title("Prediction")

In [ ]:
fig.tight_layout()
fig.savefig("../docs/toy_problem.png")